# **Methods**

## **Data Sources**

To construct a unified and comprehensive drug synonym database, we integrated four independent data sources:

1. **KatDB Synonyms**
   - **Source**: Developed by Kat Koler  
   - **File**: `L1000_BRD_name_translated_drug_list.csv`  
   - **Description**: This dataset includes multiple synonyms per BROAD compound identifier, designed to improve drug name recognition in L1000 experiments.  
   - **Example**: The BROAD_drug_ID `BRD-K52256627` includes synonyms ranging from “chlorhexidine” to complex chemical names.

2. **Therapeutic Targets Database (TTD)**
   - **URL**: [TTD Full Download](https://idrblab.org/ttd/)  
   - **File**: `P1-04-Drug_synonyms.txt`  
   - **Description**: Drug synonym lists associated with therapeutic targets.  
   - **Example**: Drug ID `D00AAN` includes synonyms like “d00aan” and various chemical descriptors.

3. **PRISM Drug Synonyms**
   - **URL**: [PRISM GitHub Repository](https://github.com/broadinstitute/prism_repurposing)  
   - **File**: `PRISM_drug_synonyms.csv`  
   - **Description**: From PRISM repurposing screen, supports MOA enrichment.  
   - **Example**: `PubChem_CID 11314340` has synonyms such as “a-674563”.

4. **LINCS 2020 Compound Metadata**
   - **File**: `compoundinfo_beta.txt`  
   - **URL**: [Clue.io Data Dashboard](https://clue.io/releases/data-dashboard)  
   - **Path**: `/data/work/tolga/Signature_processing_and_drug_prioritisation/analysis/synonym_database/input/LINCS2020/`  
   - **Description**: Contains compound identifiers used in the L1000 assay from Clue.io.

---

## **Data Characteristics**

- **Initial Entry Counts**:  
  - `katdb_df`: 13,176  
  - `ttd_df`: 299,047  
  - `prism_df`: 112,784

- **Initial Unique Synonym Counts**:  
  - BROAD: 13,175  
  - TTD: 299,046  
  - PubChem: 112,675

- **Unique Identifier Counts**:  
  - BROAD_drug_IDs: 5,539  
  - TTD_drug_IDs: 30,713  
  - PubChem_CIDs: 1,351

- **LINCS2020 Coverage**:  
  - Unique BROAD_drug_IDs: 33,613  
  - Unique synonyms: 34,234

- **Combined LINCS2020 + KatDB**:  
  - Unique BROAD_drug_IDs: 33,858  
  - Unique synonyms: 45,617

---

## **Data Preparation**

Normalization steps applied:

- Converted all synonyms to lowercase  
- Split multi-synonym strings into separate rows  
- Stripped whitespace and formatting artifacts  

This allowed consistent matching across datasets.

---

## **Merging Strategy**

1. **Synonym Explosion**  
   One synonym per row using `.explode()`.

2. **Outer Join on Synonyms**  
   To retain as many matches as possible.

3. **ID Propagation**  
   Shared synonyms helped propagate IDs across datasets using forward/backward fill.

4. **Grouping & Aggregation**  
   Aggregated synonyms based on unique sets of IDs.

5. **Filter Rows**  
   Dropped rows with missing `BROAD_drug_ID`, as the focus was on LINCS platform.

---

# **Results**

## **Merged Dataset Overview**

- **Final DataFrame Shape**: (193,221 rows × 4 columns)  
- **Final Unique Synonym Count**: 193,113  

### **Unique Identifier Counts**

- BROAD_drug_IDs: 33,858  
- TTD_drug_IDs: 2,775  
- PubChem_CIDs: 950

### **Missing Values (NaNs)**

- `BROAD_drug_ID`: 0  
- `synonyms`: 0  
- `TTD_drug_ID`: 45,737  
- `PubChem_CID`: 88,237

### **Data Types**

- BROAD_drug_ID: object  
- synonyms: object  
- TTD_drug_ID: object  
- PubChem_CID: float64

- **Duplicate Rows**: 0

---

## **Sample Entries**

### First Rows

| BROAD_drug_ID | Synonym                     | TTD_drug_ID | PubChem_CID |
| ------------- | --------------------------- | ----------- | ------------|
| BRD-K52256627 | chlorhexidine               | D0V4GY      | 9552079     |
| BRD-K52256627 | chlorhexidine, combinations | D0V4GY      | 9552079     |
| BRD-K52256627 | 1,1'-hexamethylenebis[...]  | D0V4GY      | 9552079     |

### Last Rows

| BROAD_drug_ID | Synonym                    | TTD_drug_ID | PubChem_CID |
| ------------- | -------------------------- | ----------- | ------------|
| BRD-K63068307 | 2-(difluoromethyl)-1-[...] | D0O1LD      | 11647372    |
| BRD-K63068307 | zs4                        | D0O1LD      | 11647372    |

---

## **Match Statistics**

| Category                         | Initial Entries | Final Entries |
| -------------------------------- | --------------- | ------------- |
| BROAD_drug_ID                    | 45,617          | 33,858        |
| TTD_drug_ID                      | 299,047         | 2,775         |
| PubChem_CID                      | 112,784         | 950           |
| **Total Merged Rows**            | —               | 435,530       |
| **Final Unique Synonym Groups**  | —               | 193,221       |
| **Matched Identifier Instances** | —               | 1,145         |


In [120]:
# prompt: get tsv and csv files from input folder KatDB/  PRISM_drug_synonims/  Therapeutic_Targets_Database_Synonims/ upload ['P1-04-Drug_synonyms.tsv']
# ['L1000_BRD_name_translated_drug_list.csv', 'PRISM_drug_synonyms.csv'] as ttd_df, prism_df and katdb_df

import pandas as pd
katdb_df = pd.read_csv('../input/KatDB/L1000_BRD_name_translated_drug_list.csv')

prism_df = pd.read_csv('../input/PRISM_drug_synonims/PRISM_drug_synonyms.csv')
ttd_df = pd.read_csv("../input/Therapeutic_Targets_Database_Synonims/P1-04-Drug_synonyms.tsv", sep='\t')
genesetMetadata = pd.read_csv(
    "/data/work/tolga/Signature_processing_and_drug_prioritisation/analysis/synonym_database/input/LINCS2020/compoundinfo_beta.txt",
    sep="\t",  # Tab-delimited file
    engine='c'  # Default engine should work fine for tab-delimited files
)

genesetMetadata2 = genesetMetadata

In [121]:
genesetMetadata = genesetMetadata.rename(columns={'pert_id': 'BROAD_drug_ID'})# fill NaN values in cmap_name and alias_list with empty strings

# Keep only the two required columns
genesetMetadata = genesetMetadata[['BROAD_drug_ID', 'compound_aliases']]

# Drop rows where alias_list is NaN
genesetMetadata = genesetMetadata.dropna(subset=['compound_aliases'])

genesetMetadata = genesetMetadata.explode('compound_aliases')
# Convert alias_list to lowercase and strip whitespace
genesetMetadata['compound_aliases'] = genesetMetadata['compound_aliases'].str.lower().str.strip()
# Remove duplicates in alias_list
genesetMetadata = genesetMetadata.drop_duplicates(subset=['compound_aliases'])
# Rename alias_list to synonyms
genesetMetadata = genesetMetadata.rename(columns={'compound_aliases': 'synonyms'})
# Reset index after dropping duplicates
genesetMetadata.reset_index(drop=True, inplace=True)
# Display the first 20 rows of the updated genesetMetadata DataFrame


genesetMetadata2.head(1)
#merge cmap_name and alias_list columns in genesetMetadata to synonyms
genesetMetadata2['synonyms'] = genesetMetadata2['cmap_name']
# #append alias_list to synonyms, if alias_list is not NaN
# genesetMetadata2['synonyms'] = genesetMetadata2.apply(
#     lambda row: row['synonyms'] + '|' + row['alias_list'] if pd.notna(row['alias_list']) else row['synonyms'], axis=1
# )
#rename pert_id column in genesetMetadata to BROAD_drug_ID 
genesetMetadata2 = genesetMetadata2.rename(columns={'pert_id': 'BROAD_drug_ID'})# fill NaN values in cmap_name and alias_list with empty strings

#only include BROAD_drug_ID, synonyms columns in genesetMetadata
genesetMetadata2 = genesetMetadata2[['BROAD_drug_ID', 'synonyms']]
# drop rows where synonyms is NaN
genesetMetadata2 = genesetMetadata2.dropna(subset=['synonyms'])

genesetMetadata2

lincs_df = pd.concat([genesetMetadata2, genesetMetadata], ignore_index=True)
# Remove duplicates based on the 'synonyms' column
lincs_df = lincs_df.drop_duplicates(subset=['synonyms'])
# unique Broad_drug_IDs in lincs_df
unique_broad_drug_ids = lincs_df['BROAD_drug_ID'].unique()
print(f"Number of unique BROAD_drug_IDs in LINCS2020: {len(unique_broad_drug_ids)}")
#unique synonyms in lincs_df
unique_synonyms = lincs_df['synonyms'].unique()
print(f"Number of unique synonyms in LINCS2020: {len(unique_synonyms)}")
#number if na values in synonyms column
na_synonyms_count = lincs_df['synonyms'].isna().sum()
print(f"Number of NaN values in synonyms column: {na_synonyms_count}")

Number of unique BROAD_drug_IDs in LINCS2020: 33613
Number of unique synonyms in LINCS2020: 34234
Number of NaN values in synonyms column: 0


In [122]:
# prompt: katdb_df str.lower(target_name) and drop target_name duplicates, change target_name column name to synonyms, drop input_type column, change input_name column name to BROAD_drug_ID

katdb_df['target_name'] = katdb_df['target_name'].str.lower()
katdb_df.drop_duplicates(subset=['target_name'], inplace=True)
katdb_df.rename(columns={'target_name': 'synonyms', 'input_name': 'BROAD_drug_ID'}, inplace=True)
katdb_df.drop(columns=['input_type'], inplace=True)

katdb_df.drop(columns=['target_type'], inplace=True)

katdb_df.drop(columns=['kat_id'], inplace=True)

#drop rows where synonyms is NaN in katdb_df
katdb_df = katdb_df.dropna(subset=['synonyms'])
#number of unique broad ids katdb_df
unique_broad_ids_katdb = katdb_df['BROAD_drug_ID'].unique()
print(f"Number of unique BROAD_drug_IDs in KatDB: {len(unique_broad_ids_katdb)}")
#number of unique synonyms katdb_df
unique_synonyms_katdb = katdb_df['synonyms'].unique()
print(f"Number of unique synonyms in KatDB: {len(unique_synonyms_katdb)}")



Number of unique BROAD_drug_IDs in KatDB: 5539
Number of unique synonyms in KatDB: 13175


In [123]:
# prompt: ttd_df str.lower(Synonyms_column) and drop Synonyms_column duplicates

ttd_df['Synonyms_column'] = ttd_df['Synonyms_column'].str.lower()
ttd_df = ttd_df.drop_duplicates(subset=['Synonyms_column'])
ttd_df = ttd_df.rename(columns={'Synonyms_column': 'synonyms'})
ttd_df.drop(columns=['Abbreviation'], inplace=True)

# number of unique broad ids ttd_df
unique_broad_ids_ttd = ttd_df['TTD_drug_ID'].unique()
print(f"Number of unique TTD_drug_ID in TTD: {len(unique_broad_ids_ttd)}")
# number of unique synonyms ttd_df
unique_synonyms_ttd = ttd_df['synonyms'].unique()
print(f"Number of unique synonyms in TTD: {len(unique_synonyms_ttd)}")


Number of unique TTD_drug_ID in TTD: 30713
Number of unique synonyms in TTD: 299047


In [124]:
# Function to split, lower case, and remove duplicates
def process_synonyms(synonyms):
    unique_synonyms = list(dict.fromkeys([syn.strip().lower() for syn in synonyms.split('|')]))
    return unique_synonyms

# Apply the function to the PubChem_synonyms column and explode into separate rows
prism_df['Processed_synonyms'] = prism_df['PubChem_synonyms'].apply(process_synonyms)
prism_exploded = prism_df.explode('Processed_synonyms')

# Rename the exploded column to PubChem_synonym
prism_exploded = prism_exploded.rename(columns={'Processed_synonyms': 'PubChem_synonym'})

# Drop the original PubChem_synonyms column if desired
prism_df = prism_exploded.drop(columns=['PubChem_synonyms'])
prism_df = prism_df.rename(columns={'PRISM_drug_name': 'PRISM_drug_ID'})

prism_df = prism_df.rename(columns={'PubChem_synonym': 'synonyms'})

# Display the final dataframe
prism_df.drop(columns=['PRISM_drug_ID'], inplace=True)

#number of unique PRISM_drug_ID in prism_df
unique_prism_drug_ids = prism_df['PubChem_CID'].unique()
print(f"Number of unique PubChem_CID in PRISM: {len(unique_prism_drug_ids)}")

#number of unique PubChem_synonyms in prism_df
unique_pubchem_synonyms = prism_df['synonyms'].unique()
print(f"Number of unique synonyms in PRISM: {len(unique_pubchem_synonyms)}")


Number of unique PubChem_CID in PRISM: 1351
Number of unique synonyms in PRISM: 112675


In [147]:
# prompt: print the the heads of katdb_df, ttd_df and prism_df, include their names

print("katdb_df head:")
print(katdb_df.head(1))
print("lincs_df head:")
print(lincs_df.head(1))
print("\nttd_df head:")
print(ttd_df.head(1))
print("\nprism_df head:")
print(prism_df.head(1))
# Replace "nan" strings with np.nan in ID columns of each dataframe
katdb_df['BROAD_drug_ID'].replace("nan", np.nan, inplace=True)
lincs_df['BROAD_drug_ID'].replace("nan", np.nan, inplace=True)
ttd_df['TTD_drug_ID'].replace("nan", np.nan, inplace=True)
prism_df['PubChem_CID'].replace("nan", np.nan, inplace=True)

katdb_df head:
   BROAD_drug_ID       synonyms
0  BRD-K52256627  chlorhexidine
lincs_df head:
   BROAD_drug_ID    synonyms
0  BRD-A08715367  L-theanine

ttd_df head:
  TTD_drug_ID synonyms
0      D00AAN   d00aan

prism_df head:
   PubChem_CID  synonyms
0     11314340  a-674563


In [148]:
# concatenate katdb_df and lincs_df
combined_lincs_df = pd.concat([katdb_df, lincs_df], ignore_index=True)
#drop duplicates based on synonyms column
combined_lincs_df = combined_lincs_df.drop_duplicates(subset=['synonyms'])
#number of unique BROAD_drug_IDs in combined_lincs_df
unique_broad_drug_ids_combined = combined_lincs_df['BROAD_drug_ID'].unique()
print(f"Number of unique BROAD_drug_IDs in combined LINCS2020 and KatDB: {len(unique_broad_drug_ids_combined)}")
#number of unique synonyms in combined_lincs_df
unique_synonyms_combined = combined_lincs_df['synonyms'].unique()
print(f"Number of unique synonyms in combined LINCS2020 and KatDB: {len(unique_synonyms_combined)}")

Number of unique BROAD_drug_IDs in combined LINCS2020 and KatDB: 33858
Number of unique synonyms in combined LINCS2020 and KatDB: 45617


In [149]:
from functools import reduce
dfs = [combined_lincs_df, ttd_df, prism_df]
# merged_df = reduce(lambda left, right: pd.merge(left, right, on='synonyms', how='outer'), dfs)

# Calculate shapes and unique occurrences before merging
initial_shapes = {df.columns[0]: df.shape for df in dfs}
initial_unique_counts = {df.columns[0]: df['synonyms'].nunique() for df in dfs}

# Merge the dataframes
merged_df = reduce(lambda left, right: pd.merge(left, right, on='synonyms', how='outer'), dfs)

# Calculate shape and unique occurrences after merging
final_shape = merged_df.shape
final_unique_count = merged_df['synonyms'].nunique()

# Generate the report
print("Match Report:")
print("-" * 20)
print("\nInitial Shapes:")
for name, shape in initial_shapes.items():
  print(f"{name}: {shape}")

print("\nInitial Unique Counts:")
for name, count in initial_unique_counts.items():
  print(f"{name}: {count}")

print("\nFinal Shape (Merged):", final_shape)
print("Final Unique Count (Merged):", final_unique_count)

# Calculate matched instances
matched_instances = merged_df.dropna()
print("\nMatched Instances:", matched_instances.shape[0])


Match Report:
--------------------

Initial Shapes:
BROAD_drug_ID: (45617, 2)
TTD_drug_ID: (299047, 2)
PubChem_CID: (112784, 2)

Initial Unique Counts:
BROAD_drug_ID: 45617
TTD_drug_ID: 299046
PubChem_CID: 112675

Final Shape (Merged): (435530, 4)
Final Unique Count (Merged): 435420

Matched Instances: 1145


In [151]:
# prompt: # Get the number of  unique values for each ID column in each separate DataFrame
unique_combined_lincs_df_ids = combined_lincs_df['BROAD_drug_ID'].unique()
unique_ttd_ids = ttd_df['TTD_drug_ID'].unique()
unique_prism_ids = prism_df['PubChem_CID'].unique()

# Get the number of unique values for each ID column in each separate DataFrame
num_unique_combined_lincs_df_ids = len(unique_combined_lincs_df_ids)
num_unique_ttd_ids = len(unique_ttd_ids)
num_unique_prism_ids = len(unique_prism_ids)

print("Number of unique BROAD_drug_IDs:", num_unique_combined_lincs_df_ids)
print("Number of unique TTD_drug_IDs:", num_unique_ttd_ids)
print("Number of unique PubChem_CIDs:", num_unique_prism_ids)


Number of unique BROAD_drug_IDs: 33858
Number of unique TTD_drug_IDs: 30713
Number of unique PubChem_CIDs: 1351


In [153]:
# merged df astype str
merged_df = merged_df.astype(str)

In [160]:
# check if there are any NaN values in the merged_df as "nan" strings in the ID columns
import numpy as np
nan_in_merged_df = merged_df.isin(['nan']).any().any()
if nan_in_merged_df:
    print("There are 'nan' strings in the merged DataFrame.")
else:
    print("There are no 'nan' strings in the merged DataFrame.")

There are no 'nan' strings in the merged DataFrame.


In [159]:
#There are 'nan' strings in the merged DataFrame, so we will replace them with np.nan
merged_df.replace("nan", np.nan, inplace=True)


In [ ]:
def clean_fake_nans(df, columns):
    for col in columns:
        df[col] = df[col].apply(
            lambda x: np.nan if isinstance(x, str) and x.strip().lower() in ['nan', 'missing'] else x
        )
    return df

merged_df = clean_fake_nans(merged_df, ['PubChem_CID', 'TTD_drug_ID', 'BROAD_drug_ID'])


Empty DataFrame
Columns: [PubChem_CID, TTD_drug_ID, BROAD_drug_ID]
Index: []


In [167]:
merged_df['PubChem_CID'] = pd.to_numeric(merged_df['PubChem_CID'], errors='coerce')


In [ ]:
# import pandas as pd
# import numpy as np

# # Fill NaNs in each column using grouped non-null values based on shared identifiers
# def groupwise_fill(df, group_keys, fill_columns):
#     for col in fill_columns:
#         for key in group_keys:
#             df[col] = df.groupby(key)[col].transform(lambda x: x.ffill().bfill() if x.notna().any() else x)
#     return df

# # Example usage


# #synonym_df = groupwise_fill(merged_df, group_keys, fill_columns)

In [ ]:
#synonym_df = pd.read_csv('/data/work/tolga/Signature_processing_and_drug_prioritisation/analysis/synonym_database/output/merged_40K_drug_synonyms_filled.csv')


In [169]:
group_keys = ['PubChem_CID', 'TTD_drug_ID', 'BROAD_drug_ID']  # identifiers to match on
fill_columns = ['BROAD_drug_ID', 'TTD_drug_ID', 'PubChem_CID']  # columns to fill

print("Group keys:", group_keys)
print("Fill columns:", fill_columns)

def groupwise_fill(df, group_keys, fill_columns):
    df = df.copy()
    for col in fill_columns:
        for key in group_keys:
            print(f"Processing fill column: {col} grouped by key: {key}")
            # Skip if col == key (makes no sense to fill BROAD_drug_ID by grouping on BROAD_drug_ID)
            if col == key:
                continue
            
            # Only operate where group key is not null
            valid_rows = df[key].notna()
            
            # Apply transform only on rows with valid group keys
            try:
                filled = df.loc[valid_rows].groupby(key, group_keys=False)[col].transform(
                    lambda x: x.ffill().bfill() if x.notna().any() else x
                )
                # Assign the filled values back
                df.loc[valid_rows, col] = filled
            except Exception as e:
                print(f"⚠️ Failed to groupby {key} for column {col}: {e}")
                continue
    return df



def iterative_groupwise_fill(df, group_keys, fill_columns, max_iter=5):
    df = df.copy()
    for _ in range(max_iter):
        before_fill = df[fill_columns].isna().sum().sum()
        df = groupwise_fill(df, group_keys, fill_columns)
        after_fill = df[fill_columns].isna().sum().sum()
        if after_fill >= before_fill:
            break  # Stop if no more progress
    return df

# Perform the iterative groupwise fill
synonym_df_filled = iterative_groupwise_fill(merged_df, group_keys, fill_columns)

Group keys: ['PubChem_CID', 'TTD_drug_ID', 'BROAD_drug_ID']
Fill columns: ['BROAD_drug_ID', 'TTD_drug_ID', 'PubChem_CID']
Processing fill column: BROAD_drug_ID grouped by key: PubChem_CID
Processing fill column: BROAD_drug_ID grouped by key: TTD_drug_ID
Processing fill column: BROAD_drug_ID grouped by key: BROAD_drug_ID
Processing fill column: TTD_drug_ID grouped by key: PubChem_CID
Processing fill column: TTD_drug_ID grouped by key: TTD_drug_ID
Processing fill column: TTD_drug_ID grouped by key: BROAD_drug_ID
Processing fill column: PubChem_CID grouped by key: PubChem_CID
Processing fill column: PubChem_CID grouped by key: TTD_drug_ID
Processing fill column: PubChem_CID grouped by key: BROAD_drug_ID
Processing fill column: BROAD_drug_ID grouped by key: PubChem_CID
Processing fill column: BROAD_drug_ID grouped by key: TTD_drug_ID
Processing fill column: BROAD_drug_ID grouped by key: BROAD_drug_ID
Processing fill column: TTD_drug_ID grouped by key: PubChem_CID
Processing fill column: TT

In [170]:
synonym_df_filled

,BROAD_drug_ID,synonyms,TTD_drug_ID,PubChem_CID
0,BRD-K52256627,chlorhexidine,D0V4GY,9552079.0
1,BRD-K52256627,"chlorhexidine, combinations",D0V4GY,9552079.0
2,BRD-K52256627,"1,1'-hexamethylenebis[5-(4-chlorophenyl)biguan...",D0V4GY,9552079.0
3,BRD-K52256627,1n-[6-[4-chloroanilino(imino)methylamino(imino...,D0V4GY,9552079.0
4,BRD-K52256627,biguanide derivative,D0V4GY,9552079.0
...,...,...,...,...
435525,BRD-K63068307,"2-(difluoromethyl)-1-[4,6-di(4-morpholinyl)-1,...",D0O1LD,11647372.0
435526,BRD-K63068307,"2-(difluoromethyl)-1-[4,6-di(morpholin-4-yl)-1...",D0O1LD,11647372.0
435527,BRD-K63068307,"4,4''-(6-(2-(difluoromethyl)-1h-benzo[d]imidaz...",D0O1LD,11647372.0
435528,BRD-K63068307,"4,4-(6-(2-(difluoromethyl)-1h-benzo[d]imidazol...",D0O1LD,11647372.0


In [172]:
synonym_df_filled = iterative_groupwise_fill(synonym_df_filled, group_keys, fill_columns)

Processing fill column: BROAD_drug_ID grouped by key: PubChem_CID
Processing fill column: BROAD_drug_ID grouped by key: TTD_drug_ID
Processing fill column: BROAD_drug_ID grouped by key: BROAD_drug_ID
Processing fill column: TTD_drug_ID grouped by key: PubChem_CID
Processing fill column: TTD_drug_ID grouped by key: TTD_drug_ID
Processing fill column: TTD_drug_ID grouped by key: BROAD_drug_ID
Processing fill column: PubChem_CID grouped by key: PubChem_CID
Processing fill column: PubChem_CID grouped by key: TTD_drug_ID
Processing fill column: PubChem_CID grouped by key: BROAD_drug_ID


In [ ]:

#synonym_df_filled = pd.read_csv('../output/merged_200K_drug_synonyms_w_sigmeta.csv')

In [175]:

#get statistics of synonym_df_filled
synonym_df_filled.describe(include='all')
# Check for NaN values in the DataFrame
nan_counts = synonym_df_filled.isna().sum()
print("NaN counts in each column:")
print(nan_counts)
# Check for unique values in each column
unique_counts = synonym_df_filled.nunique()
print("Unique counts in each column:")
print(unique_counts)
# Check the data types of each column
data_types = synonym_df_filled.dtypes
print("Data types of each column:")
print(data_types)
# Check the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(synonym_df_filled.head())
# Check the last few rows of the DataFrame
print("Last few rows of the DataFrame:")
print(synonym_df_filled.tail())
# Check the shape of the DataFrame
print("Shape of the DataFrame:", synonym_df_filled.shape)
# Check for duplicate rows in the DataFrame
duplicate_rows = synonym_df_filled.duplicated().sum()
print("Number of duplicate rows:", duplicate_rows)
# Check for missing values in the DataFrame
missing_values = synonym_df_filled.isnull().sum()
print("Missing values in each column:")
print(missing_values)



NaN counts in each column:
BROAD_drug_ID    242309
synonyms              1
TTD_drug_ID       51795
PubChem_CID      310169
dtype: int64
Unique counts in each column:
BROAD_drug_ID     33858
synonyms         435420
TTD_drug_ID       30713
PubChem_CID        1351
dtype: int64
Data types of each column:
BROAD_drug_ID     object
synonyms          object
TTD_drug_ID       object
PubChem_CID      float64
dtype: object
First few rows of the DataFrame:
   BROAD_drug_ID                                           synonyms  \
0  BRD-K52256627                                      chlorhexidine   
1  BRD-K52256627                        chlorhexidine, combinations   
2  BRD-K52256627  1,1'-hexamethylenebis[5-(4-chlorophenyl)biguan...   
3  BRD-K52256627  1n-[6-[4-chloroanilino(imino)methylamino(imino...   
4  BRD-K52256627                               biguanide derivative   

  TTD_drug_ID  PubChem_CID  
0      D0V4GY    9552079.0  
1      D0V4GY    9552079.0  
2      D0V4GY    9552079.0  
3      D

In [ ]:
# drop duplicates in synonym_df_filled
synonym_df_filled = synonym_df_filled.drop_duplicates()
# reset index of synonym_df_filled
synonym_df_filled.reset_index(drop=True, inplace=True)
# Save the cleaned DataFrame to a new CSV file
#drop na rows synoyms
synonym_df_filled = synonym_df_filled.dropna(subset=['synonyms'])
synonym_df_filled = synonym_df_filled.dropna(subset=['BROAD_drug_ID'])


In [177]:

#get statistics of synonym_df_filled
synonym_df_filled.describe(include='all')
# Check for NaN values in the DataFrame
nan_counts = synonym_df_filled.isna().sum()
print("NaN counts in each column:")
print(nan_counts)
# Check for unique values in each column
unique_counts = synonym_df_filled.nunique()
print("Unique counts in each column:")
print(unique_counts)
# Check the data types of each column
data_types = synonym_df_filled.dtypes
print("Data types of each column:")
print(data_types)
# Check the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(synonym_df_filled.head())
# Check the last few rows of the DataFrame
print("Last few rows of the DataFrame:")
print(synonym_df_filled.tail())
# Check the shape of the DataFrame
print("Shape of the DataFrame:", synonym_df_filled.shape)
# Check for duplicate rows in the DataFrame
duplicate_rows = synonym_df_filled.duplicated().sum()
print("Number of duplicate rows:", duplicate_rows)
# Check for missing values in the DataFrame
missing_values = synonym_df_filled.isnull().sum()
print("Missing values in each column:")
print(missing_values)



NaN counts in each column:
BROAD_drug_ID        0
synonyms             0
TTD_drug_ID      45737
PubChem_CID      88237
dtype: int64
Unique counts in each column:
BROAD_drug_ID     33858
synonyms         193113
TTD_drug_ID        2775
PubChem_CID         950
dtype: int64
Data types of each column:
BROAD_drug_ID     object
synonyms          object
TTD_drug_ID       object
PubChem_CID      float64
dtype: object
First few rows of the DataFrame:
   BROAD_drug_ID                                           synonyms  \
0  BRD-K52256627                                      chlorhexidine   
1  BRD-K52256627                        chlorhexidine, combinations   
2  BRD-K52256627  1,1'-hexamethylenebis[5-(4-chlorophenyl)biguan...   
3  BRD-K52256627  1n-[6-[4-chloroanilino(imino)methylamino(imino...   
4  BRD-K52256627                               biguanide derivative   

  TTD_drug_ID  PubChem_CID  
0      D0V4GY    9552079.0  
1      D0V4GY    9552079.0  
2      D0V4GY    9552079.0  
3      D0V4G

In [178]:
synonym_df_filled.to_csv('../output/merged_200K_drug_synonyms.csv', index=False, compression=None)